In [1]:
# Imports

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import logging
import numpy as np
import os

import tensorflow as tf
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

/home/pmirallesr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pmirallesr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pmirallesr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pmirallesr/.local/lib/python3.6/site-packages/tensorflow/python/framework

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
nbEpochs = 50
train_batch_size = 100
test_batch_size = 100
source = "SVHN"
target = "MNIST"
sourceChannels = 1
dataPath = os.path.dirname(os.getcwd()) + "/data/"
randomSeed = 1905
print(dataPath)

/home/pmirallesr/eclipse-workspace/DRCN-Torch/data/


In [4]:
np.random.seed(randomSeed)
torch.manual_seed(randomSeed)

**TEXT**

In [5]:
class GaussianDenoising:
    """Distort a pixel with additive [+ N(0,scale)] or multiplicative [x N(1,scale)] gaussian noise"""

    def __init__(self, sigma = 0.2, effectType = "additive"):
        self.sigma = sigma
        self.effectType = effectType

    def __call__(self, x):
        if effectType == "multiplicative":
            return x.numpy() * np.random.normal(loc = 1.0, scale = sigma, size = x.shape)
        elif effectType == "additive":
            return x.numpy + np.random.normal(loc = 0.0, scale = sigma, size = x.shape)
        else:
            print("Specify a valid type of gaussian error: multiplicative or additive")
            raise ValueError

In [6]:
class ImpulseDenoising:
    """Erase a pixel with probability p"""

    def __init__(self, p = 0.5):
        self.p = p

    def __call__(self, x):
        return x.numpy() * np.random.binomial(1, self.p, size=x.shape)

In [7]:
# Data Transforms
# Normalizations commented as torch already imports normalized data on 32x32

# Data Augmentation - Geometric Transformations
# 20º random rotation
# 20% random height and width shifts
dataAugmentTransforms = []
dataAugmentTransforms.append(torchvision.transforms.RandomAffine(degrees = 20, translate = (0.2, 0.2)))
# Denoising
dataAugmentTransforms.append(ImpulseDenoising())

MNIST_Transforms = []
MNIST_Transforms.append(torchvision.transforms.ToTensor())
#MNIST_Transforms.append(torchvision.transforms.Resize((32,32)))
# MNIST_Mean = 0.1307
# MNIST_StDev = 0.3081
# MNIST_Transforms.append(torchvision.transforms.Normalize(MNIST_Mean, MNIST_StDev))



SVHN_Transforms = []
SVHN_Transforms.append(torchvision.transforms.Grayscale())
SVHN_Transforms.append(torchvision.transforms.ToTensor())
# SVHN_Mean = 0.4657
# SVHN_StDev = 0.2025
# SVHN_Transforms.append(torchvision.transforms.Normalize(SVHN_Mean, SVHN_StDev))



In [8]:
train_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = True, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms + dataAugmentTransforms)), batch_size = train_batch_size, shuffle = True)

test_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = False, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms)), batch_size = test_batch_size, shuffle = True)

#DataLoader has irregular behaviour, does not autom create an SVHN folder but does so for MNIST
train_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "train", download = True, \
                transform = torchvision.transforms.Compose\
                (SVHN_Transforms)), batch_size = train_batch_size, shuffle = True)

test_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "test", download = True , \
                 transform = torchvision.transforms.Compose \
                (SVHN_Transforms + dataAugmentTransforms)), batch_size = test_batch_size, shuffle = True)


100.1%

Extracting /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw/train-images-idx3-ubyte.gz to /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw


113.5%

Extracting /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw/train-labels-idx1-ubyte.gz to /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw


100.4%

Extracting /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw


180.4%/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Extracting /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/MNIST/raw
Processing...
Done!


100.0%

100.0%

In [9]:
# Models

# Move function to some utils module
def calcPoolOutputSize(inputSize, kernelSize):
    return int(inputSize/(kernelSize[0]*kernelSize[1]))

       
class Encoder(nn.Module):
    """Encoder common to Autoencoder and labeller"""

    def __init__(self, inputChannels):
        """Initialize DomainRegressor."""
        super(Encoder, self).__init__()
        
        #Size Parameters
        
        conv1OutputSize = 100
        conv1KernelSize = 5
        
        maxPool1Size = (2,2)
        
        conv2OutputSize = 150
        conv2KernelSize = 5
        
        maxPool2Size = (2,2)
        
        conv3OutputSize = 200
        conv3KernelSize = 3
        
        # Placeholder ranges
#         fc4OutputSize = range(300,1000,50)
#         fc5OutputSize = range(300,1000,50)
        fc4OutputSize = 300
        fc5OutputSize = 300
        
        
        # Size Calculations
        conv1InputSize = inputChannels
        
        conv2InputSize = calcPoolOutputSize(conv1OutputSize,maxPool1Size)
        
        conv3InputSize = calcPoolOutputSize(conv2OutputSize,maxPool2Size)
        
        fc4InputSize = conv3OutputSize
        
        fc5InputSize = fc4OutputSize
        
        # Layers
        
        self.conv1 = nn.Conv2d(conv1InputSize, conv1OutputSize, conv1KernelSize)
        self.maxPool2D1 = nn.MaxPool2d(maxPool1Size)
        self.conv2 = nn.Conv2d(conv2InputSize, conv2OutputSize, conv2KernelSize)
        self.maxPool2D2 = nn.MaxPool2d(maxPool2Size)
        self.conv3 = nn.Conv2d(conv3InputSize, conv3OutputSize, conv3KernelSize)
        
        self.fc4 = nn.Linear(fc4InputSize, fc4OutputSize)
        self.fc5 = nn.Linear(fc5InputSize, fc5OutputSize)
        


    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = F.relu(self.conv1(x))
        x = self.maxPool2D1(x)
        
        x = F.relu(self.conv2(x))
        x = self.maxPool2D2(x)
        
        x = F.relu(self.conv3(x))
        x = self.maxPool2D3(x)
        
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x

class Labeller(nn.Module):
    """ The labeller part of the network is constituted by 
    the common Encoder plus a labelling fully connected layer"""
    def __init__(self, encoder):
        super(Labeller, self).__init__()
        self.encoder = encoder
        fc3OutputSize = 10 # for 10 possible digits
        fc3InputSize = self.encoder.fc5.out_features
        self.fcOUT = nn.Linear(fc3InputSize, fc3OutputSize)  
        
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(self.fcOUT(x))

    
class Autoencoder(nn.Module):
    """The autoencoder is constituted by the Encoder common to
    the labeller and itself, and a decoder part that is a mirror
    image of the Encoder
    
    Layers 6 and 7 are FC layers, layers 8 through 10 are (de)convolutional layers
    
    """

    def __init__(self, encoder):
        """Initialize DomainRegressor."""
        super(Autoencoder, self).__init__()
        
        self.encoder = encoder
        #Size Parameters
        
        deconv8InputSize = 100
        deconv8KernelSize = 5
        
        upscale8Size = self.encoder.maxPool2D2.kernel_size
        
        deconv9InputSize = 150
        deconv9KernelSize = 5
        
        upscale9Size = self.encoder.maxPool2D1.kernel_size
        
        deconv10InputSize = 200
        deconv10KernelSize = 3
        
        # Placeholder ranges - substitute for actual numbers for usage
#         fc7OutputSize = range(300,1000,50)
#         fc6OutputSize = range(300,1000,50)
        fc7OutputSize = 300
        fc6OutputSize = 300
        
        
        # Size Calculations
        deconv8OutputSize = self.encoder.conv3.in_channels
        deconv9OutputSize = self.encoder.conv2.in_channels
        deconv10OutputSize = self.encoder.conv1.in_channels
        
        #Is calcPoolSize still valid for Unspooling?
        
#         deconv9OutputSize = calcPoolOutputSize(deconv8OutputSize,maxPool8Size)
        
#         deconv10OutputSize = calcPoolOutputSize(deconv9OutputSize,maxPool9Size)
        
        deconv8InputSize = fc7OutputSize
        fc6InputSize = self.encoder.fc5.out_features
        fc7InputSize = fc6OutputSize
        
        # Layers
        
        self.fc7 = nn.Linear(fc7InputSize, fc7OutputSize)
        self.fc6 = nn.Linear(fc6InputSize, fc6OutputSize)
        
        self.deconv8 = nn.Conv2d(deconv8InputSize, deconv8OutputSize, deconv8KernelSize)
        self.upsample8 = nn.Upsample(scale_factor = 2, mode = "nearest")
        self.deconv9 = nn.Conv2d(deconv9InputSize, deconv9OutputSize, deconv9KernelSize)
        self.upsample9 = nn.Upsample(scale_factor = 2, mode = "nearest")
        self.deconv10 = nn.Conv2d(deconv10InputSize, deconv10OutputSize, deconv10KernelSize)
        
        


    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = self.encoder(x)
        
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        
        x = F.relu(self.deconv8(x))
        x = F.relu(self.maxUnspool2D8(x))
        
        x = F.relu(self.deconv9(x))
        x = F.relu(self.maxUnspool2D9(x))
        
        x = self.deconv10(x)
        
       
        return x
    

In [10]:
encoder = Encoder(sourceChannels)
labeller = Labeller(encoder)
autoencoder = Autoencoder(encoder)

In [11]:
# controlPenalty = 0.4...0.7
labellerOptimizer = torch.optim.RMSprop(labeller.parameters(), lr=1e-4, alpha=0.9, eps=1e-08, weight_decay=0.9, momentum=0, centered=False)
autoencoderOptimizer = torch.optim.RMSprop(labeller.parameters(), lr=1e-4, alpha=0.9, eps=1e-08, weight_decay=0.9, momentum=0, centered=False)

In [12]:
encoder.train()
labeller.train()
labellerOptimizer.zero_grad()
autoencoderOptimizer.zero_grad()
for epoch in range(nbEpochs):
    print("Epoch {}".format(epoch))
    for batch_id, (data, target) in enumerate(train_SVHN_loader):
        print("Source batch {}".format(batch_id))
        forward = labeller(data)
        loss = nn.CrossEntropyLoss()
        loss = loss(forward, target)
        tf.summary.scalar('Labeller Loss', loss, step=epoch)
        loss.backward()
        labellerOptimizer.step()
        
    for batch_id, (data, target) in enumerate(train_MNIST_loader):
        print("Target batch {}".format(batch_id))
        forward = autoencoder(data)
        loss = nn.MSELoss()
        loss = loss(forward, target)
        tf.summary.scalar('Autoencoder Loss', loss, step=epoch)
        loss.backward()
        autoencoderOptimizer.step()
    

Epoch 0
Source batch 0


RuntimeError: Given groups=1, weight of size [150, 25, 5, 5], expected input[100, 100, 14, 14] to have 25 channels, but got 100 channels instead